In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import pandas as pd
from random import random
import json, os.path, pickle
import math

#result = pd.read_csv(f"test10.csv")
result = pd.read_csv(f"test10a.2.csv")
pairs = pd.read_csv(f"downstairs_pairs.csv")
pairs['comparision'] = pairs['pair']
result['expected_n_attempts'] = result['students_per_group']*result['attempts_per_student']*result['trials']*2
result['trial_size'] = result['students_per_group']+result['attempts_per_student']/100
pre ="geeBIs" # test prefix 
power = .8 # %
result[f'{pre}powered'] = (result[f'{pre}wrong']+result[f'{pre}ns']) / (result[f'{pre}sig']+result[f'{pre}vsig']) < (1-power)/power
result[f'powered'] = result[f'{pre}powered'] # make a generic col
print(result.columns, pairs.columns)

Index(['Unnamed: 0', 'student_set', 'comparision', 'students_per_group',
       'attempts_per_student', 'groupA', 'groupB', 'total_attempts', 'trials',
       'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp',
       'query', 'time', 'key', 'expected_n_attempts', 'trial_size',
       'geeBIspowered', 'powered'],
      dtype='object') Index(['Unnamed: 0', 'student_set', 'pair', 'ga', 'gb', 'a', 'b', 'dif',
       'comparision'],
      dtype='object')


In [2]:
pairs

,Unnamed: 0,student_set,pair,ga,gb,a,b,dif,comparision
0,0,variable,none,control,nodif,58.103500,58.197500,0.094000,none
1,1,variable,vsmall,control,interventionA,58.103500,61.784375,3.680875,vsmall
2,2,variable,small,control,interventionB,58.103500,64.887875,6.784375,small
3,3,variable,mod,control,interventionBa,58.103500,70.061875,11.958375,mod
4,4,variable,mod1,control,interventionBb,58.103500,72.029125,13.925625,mod1
5,5,variable,big,control,interventionC,58.103500,73.754125,15.650625,big
6,6,variable,huge,control,interventionD,58.103500,76.755250,18.651750,huge
7,7,variable,upper,interventionC,interventionD,73.754125,76.755250,3.001125,upper
8,8,uniform,none,control,nodif,49.964000,49.975875,0.011875,none
9,9,uniform,vsmall,control,interventionA,49.964000,54.526875,4.562875,vsmall


In [3]:
result

,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,geeBIswrong,...,geeBIssig,geeBIsvsig,geeBIsp,query,time,key,expected_n_attempts,trial_size,geeBIspowered,powered
0,0,variable,none,20,10,control,nodif,40000,100,2,...,4,0,46.766629,"group == ['control', 'nodif'] and pint < 20 an...",1.626865e+12,d.100none.20.10,40000,20.1,False,False
1,1,variable,none,20,20,control,nodif,80000,100,2,...,3,0,44.952893,"group == ['control', 'nodif'] and pint < 20 an...",1.626865e+12,d.100none.20.20,80000,20.2,False,False
2,2,variable,none,30,10,control,nodif,60000,100,3,...,1,1,46.772963,"group == ['control', 'nodif'] and pint < 30 an...",1.626864e+12,d.100none.30.10,60000,30.1,False,False
3,3,variable,none,30,20,control,nodif,120000,100,1,...,4,0,49.569228,"group == ['control', 'nodif'] and pint < 30 an...",1.626865e+12,d.100none.30.20,120000,30.2,False,False
4,4,variable,none,50,10,control,nodif,100000,100,3,...,2,0,50.954818,"group == ['control', 'nodif'] and pint < 50 an...",1.626865e+12,d.100none.50.10,100000,50.1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,107,second,upper,100,20,interventionC,interventionD,400000,100,0,...,19,9,31.748353,"group == ['interventionC', 'interventionD'] an...",1.626890e+12,d.100upper.100.20,400000,100.2,False,False
444,108,second,upper,200,10,interventionC,interventionD,400000,100,0,...,16,17,28.360248,"group == ['interventionC', 'interventionD'] an...",1.626890e+12,d.100upper.200.10,400000,200.1,False,False
445,109,second,upper,200,20,interventionC,interventionD,800000,100,0,...,20,24,18.643000,"group == ['interventionC', 'interventionD'] an...",1.626890e+12,d.100upper.200.20,800000,200.2,False,False
446,110,second,upper,400,10,interventionC,interventionD,800000,100,0,...,22,32,11.385450,"group == ['interventionC', 'interventionD'] an...",1.626890e+12,d.100upper.400.10,800000,400.1,False,False


In [4]:
ss = result['student_set'].unique()
print(ss)

['variable' 'uniform' 'initial' 'second']


In [5]:
cs = result['comparision'].unique()
print(cs)

['none' 'vsmall' 'small' 'mod' 'mod1' 'big' 'huge' 'upper']


In [6]:
print("pairs match " if (pairs['pair'].unique() == cs).all() else "pairs dont match")

pairs match 


In [7]:
result['students_per_group'].unique(), result['attempts_per_student'].unique(), 

(array([ 20,  30,  50,  75, 100, 200, 400]), array([10, 20]))

In [10]:
# Summarise the data into a nice table:
def unpack_trial_size(s,c, missing = 400.200):
    qs = f"student_set == '{s}' and comparision == '{c}'"
    packed = result.query(f"{qs} and powered")['trial_size'].min()
    x = missing if math.isnan(packed) else packed
    size = f"{'> ' if math.isnan(packed) else ''}{int(x)} x {round((x-int(x))*100)}"
    a, b, diff = round(pairs.query(qs)[['a','b','dif']].max(),0)
    #return f"{size} [{a}%-{b}% ({diff}%)]"
    return f"{size} [{a}%-{b}% ({round(100*(1 - (100-b)/(100-a)),2)} {diff}%)]"

 

x = pd.DataFrame([{'Comparison': c, 
           **{f"{s.title()} students": unpack_trial_size(s,c) for s in ss
              for s in ss}} for c in cs if c != 'none'])
print("Trial size: Students x attempts for each scenario")
x

Trial size: Students x attempts for each scenario


,Comparison,Variable students,Uniform students,Initial students,Second students
0,vsmall,> 400 x 20 [58.0%-62.0% (9.52 4.0%)],100 x 20 [50.0%-55.0% (10.0 5.0%)],200 x 20 [52.0%-56.0% (8.33 4.0%)],> 400 x 20 [75.0%-77.0% (8.0 2.0%)]
1,small,200 x 10 [58.0%-65.0% (16.67 7.0%)],50 x 20 [50.0%-58.0% (16.0 8.0%)],75 x 20 [52.0%-60.0% (16.67 8.0%)],400 x 10 [75.0%-79.0% (16.0 4.0%)]
2,mod,50 x 20 [58.0%-70.0% (28.57 12.0%)],20 x 10 [50.0%-64.0% (28.0 14.0%)],30 x 10 [52.0%-66.0% (29.17 14.0%)],75 x 20 [75.0%-82.0% (28.0 7.0%)]
3,mod1,50 x 10 [58.0%-72.0% (33.33 14.0%)],20 x 10 [50.0%-67.0% (34.0 17.0%)],20 x 20 [52.0%-68.0% (33.33 16.0%)],50 x 20 [75.0%-83.0% (32.0 8.0%)]
4,big,30 x 10 [58.0%-74.0% (38.1 16.0%)],20 x 10 [50.0%-69.0% (38.0 19.0%)],20 x 10 [52.0%-70.0% (37.5 18.0%)],50 x 10 [75.0%-84.0% (36.0 9.0%)]
5,huge,20 x 10 [58.0%-77.0% (45.24 19.0%)],20 x 10 [50.0%-72.0% (44.0 22.0%)],20 x 10 [52.0%-73.0% (43.75 21.0%)],30 x 10 [75.0%-86.0% (44.0 11.0%)]
6,upper,> 400 x 20 [74.0%-77.0% (11.54 3.0%)],200 x 20 [69.0%-72.0% (9.68 4.0%)],400 x 10 [70.0%-73.0% (10.0 3.0%)],> 400 x 20 [84.0%-86.0% (12.5 2.0%)]


In [11]:
fields = ['student_set', 'comparision', 'students_per_group',
       'attempts_per_student',
        'total_attempts', 'trials', 'expected_n_attempts',
          'groupA', 'groupB']
query = "expected_n_attempts != total_attempts"
bad = result.query(query)[fields]
if len(bad):
    print('size errors (ie from running on a sym against a set that does not contain enough rows!')
    display(bad)
else:
    print('no size errors')

no size errors


In [12]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"`{c}` < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
query = "comparision == 'small' and students_per_group in (75, 100)"
query = "comparision == 'mod' and students_per_group == 100"
print(query)
fields = ['student_set', 'comparision', 'students_per_group', 
       'attempts_per_student', 'trials',
          #'groupA', 'groupB', 'total_attempts', 'trials',
       #'geeBwrong', 'geeBns', 'geeBsig', 'geeBvsig', 'geeBp', 
       #'geeBIwrong', 'geeBIns', 'geeBIsig', 'geeBIvsig', 'geeBIp', 
       'geeBIswrong', 'geeBIsns', 'geeBIssig', 'geeBIsvsig', 'geeBIsp'
         ]
result.query(query)[fields]

Finding powered combinations where count of NS trials is less than 80%
comparision == 'mod' and students_per_group == 100


,student_set,comparision,students_per_group,attempts_per_student,trials,geeBIswrong,geeBIsns,geeBIssig,geeBIsvsig,geeBIsp
50,variable,mod,100,10,100,0,2,8,90,5.022250e-01
51,variable,mod,100,20,100,0,1,8,91,3.433378e-01
162,uniform,mod,100,10,100,0,0,0,100,1.345664e-03
163,uniform,mod,100,20,100,0,0,0,100,1.271928e-07
274,initial,mod,100,10,100,0,0,0,100,2.315328e-03
275,initial,mod,100,20,100,0,0,0,100,3.258328e-04
386,second,mod,100,10,100,0,12,22,66,2.574211e+00
387,second,mod,100,20,100,0,4,13,83,8.164139e-01


In [13]:
# Look for invalid measures:
print("These counts chould be ~2.5")
result.query("comparision == 'none'")[[c for c in result.columns if 'wrong' in c]]


These counts chould be ~2.5


,geeBIswrong
0,2
1,2
2,3
3,1
4,3
5,4
6,4
7,4
8,4
9,5


In [14]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"{c} < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
result.query(query)

Finding powered combinations where count of NS trials is less than 80%


,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,geeBIswrong,...,geeBIssig,geeBIsvsig,geeBIsp,query,time,key,expected_n_attempts,trial_size,geeBIspowered,powered
38,38,variable,small,200,10,control,interventionB,400000,100,0,...,22,66,3.006418e+00,"group == ['control', 'interventionB'] and pint...",1.626865e+12,d.100small.200.10,400000,200.1,True,True
39,39,variable,small,200,20,control,interventionB,800000,100,0,...,17,74,1.588453e+00,"group == ['control', 'interventionB'] and pint...",1.626865e+12,d.100small.200.20,800000,200.2,True,True
40,40,variable,small,400,10,control,interventionB,800000,100,0,...,4,95,1.861293e-01,"group == ['control', 'interventionB'] and pint...",1.626865e+12,d.100small.400.10,800000,400.1,True,True
41,41,variable,small,400,20,control,interventionB,1600000,100,0,...,3,97,8.459023e-02,"group == ['control', 'interventionB'] and pint...",1.626865e+12,d.100small.400.20,1600000,400.2,True,True
47,47,variable,mod,50,20,control,interventionBa,200000,100,0,...,20,62,3.905011e+00,"group == ['control', 'interventionBa'] and pin...",1.626865e+12,d.100mod.50.20,200000,50.2,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,93,second,huge,100,20,control,interventionD,400000,100,0,...,1,99,1.064912e-02,"group == ['control', 'interventionD'] and pint...",1.626890e+12,d.100huge.100.20,400000,100.2,True,True
430,94,second,huge,200,10,control,interventionD,400000,100,0,...,0,100,7.718613e-06,"group == ['control', 'interventionD'] and pint...",1.626890e+12,d.100huge.200.10,400000,200.1,True,True
431,95,second,huge,200,20,control,interventionD,800000,100,0,...,0,100,8.260717e-07,"group == ['control', 'interventionD'] and pint...",1.626890e+12,d.100huge.200.20,800000,200.2,True,True
432,96,second,huge,400,10,control,interventionD,800000,100,0,...,0,100,1.135701e-14,"group == ['control', 'interventionD'] and pint...",1.626890e+12,d.100huge.400.10,800000,400.1,True,True
